In [443]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
np.random.seed(42)

In [56]:
def init_params(n_x, n_h, n_y):
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))

    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}

    return parameters

In [202]:
def forward(X, parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    b1 = parameters['b1']
    b2 = parameters['b2']

    Z1 = np.dot(W1,X) + b1
    A1 = np.tanh(Z1)
    #A1 = 1/(1 + np.exp(-Z1))
    Z2 = np.dot(W2, A1) + b2
    A2 = 1/(1 + np.exp(-Z2))

    #print(A2.shape)
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2":A2}
    return A2, cache

In [69]:
def get_cost(A2, Y):
    m = Y.shape[1]

    cost = np.dot((Y*np.log(A2) + (1-Y)*np.log(1-A2)), np.ones((m,1)))*(-1/m)
    cost = float(np.squeeze(cost))
    return cost

In [6]:
def backward(parameters, cache, X, Y):
    m = X.shape[1]
    W1 = parameters['W1']
    W2 = parameters['W2']
    A1 = cache['A1']
    A2 = cache['A2']
    
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T)/m
    db2 = np.sum(dZ2, axis = 1, keepdims = True)/m
    dZ1 = np.dot(W2.T, dZ2)*(1 - np.power(A1,2))
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1, axis = 1, keepdims = True)/m

    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads


In [7]:
def update(parameters, grads, learning_rate = 0.001):
    W1 = copy.deepcopy(parameters['W1'])
    b1 = copy.deepcopy(parameters['b1'])
    W2 = copy.deepcopy(parameters['W2'])
    b2 = copy.deepcopy(parameters['b2'])

    dW1 = copy.deepcopy(grads['dW1'])
    db1 = copy.deepcopy(grads['db1'])
    dW2 = copy.deepcopy(grads['dW2'])
    db2 = copy.deepcopy(grads['db2'])

    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2

    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters

In [84]:
def nn_model(X,Y,n_h, iter_num=1000000,lr = 0.0002 , pr_cost = False):
    np.random.seed(42)
    n_x = X.shape[0]
    n_y = Y.shape[0]
    print(X.shape,Y.shape)
    parameters = init_params(n_x, n_h, n_y)
    for i in range(0,iter_num):
        A2, cache = forward(X, parameters)
        cost = get_cost(A2,Y)
        grads = backward(parameters, cache, X, Y)
        parameters = update(parameters, grads, learning_rate = lr)
        if pr_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [624]:
def predict(parameters, X,th):
    A2, cache = forward(X, parameters)
    predictions = (A2 > th)
    return predictions

In [663]:
def prepareData(df,df_test,is_valid = True):
    train = df.copy()
    test = df_test.copy()
    for label,content in train.items():
        pass
        #max=train[label].max()
        #train[label]=train[label].apply(lambda x: x/max)

    for label,content in test.items():
        pass
        #max=test[label].max()
        #test[label]=test[label].apply(lambda x: x/max)
    
    X_train= train.drop('Survived', axis=1)
    y_train= train.Survived.values
    y_train= y_train.reshape(-1,1)
    if is_valid:
        X_test = test.drop('Survived', axis=1)
        y_test = test.Survived.values
        y_test = y_test.reshape(-1,1)
        return X_train.T, y_train.T, X_test.T, y_test.T
    else:
        
        return X_train.T, y_train.T, test.T, np.array([0])

Import dataset

In [595]:
df = pd.read_csv('Data/train.csv')
df_test = pd.read_csv('Data/test.csv')

In [596]:
df.drop(columns='PassengerId', inplace=True)
df_test.drop(columns='PassengerId', inplace=True)

df.drop(columns='Cabin', inplace=True)
df_test.drop(columns='Cabin', inplace=True)

df.drop(columns='Ticket', inplace=True)
df_test.drop(columns='Ticket', inplace=True)

In [597]:
Passangers =  ['Mrs', 'Mr', 'Miss', 'Ms', 'Mme', 'Mlle', 'Master']
Royals = ['Lady', 'Sir', 'Don', 'Jonkheer', 'Countess', 'Dona']
Crews = ['Major', 'Col', 'Rev', 'Capt', 'Dr']

df['Title'] = df['Name'].apply(lambda x: x.split('.')[0].split()[-1])
df_test['Title'] = df_test['Name'].apply(lambda x: x.split('.')[0].split()[-1])

df['Status']= df['Title'].apply(lambda x: 'Passanger' if x in Passangers  else ('Royale' if x in Royals else ('Crew' if x in Crews else 'Missing')))
df_test['Status']= df_test['Title'].apply(lambda x: 'Passanger' if x in Passangers  else ('Royale' if x in Royals else ('Crew' if x in Crews else 'Missing')))


df.drop(columns='Name', inplace=True)
df_test.drop(columns='Name', inplace=True)

df.drop(columns='Title', inplace=True)
df_test.drop(columns='Title', inplace=True)

In [598]:
df['Embarked'].fillna('S', inplace=True)
df_test['Fare'].fillna(26.55, inplace=True)

In [599]:
for i,r in df.iterrows():
    if pd.isnull(r['Age']):
        df.loc[df.index[i],'Age'] = df[df['Pclass'] == r['Pclass']]['Age'].median()

for i,r in df_test.iterrows():
    if pd.isnull(r['Age']):
        df_test.loc[df_test.index[i],'Age'] = df_test[df_test['Pclass'] == r['Pclass']]['Age'].median()

In [554]:
df['Family']=df['Parch']+df['SibSp']+1
df_test['Family']=df_test['Parch']+df_test['SibSp']+1

In [456]:
df['Fare']=df['Fare']/df['Family']
df_test['Fare']=df_test['Fare']/df_test['Family']

In [600]:
"""df.drop(columns='Fare', inplace=True)
df_test.drop(columns='Fare', inplace=True)"""

df.drop(columns='Status', inplace=True)
df_test.drop(columns='Status', inplace=True)

In [556]:
df['Family'] = pd.cut(df['Family'], bins=[0,1,4,8,15])
df_test['Family'] = pd.cut(df_test['Family'], bins=[0,1,4,8,15])

arr=[0,5,12,20,30,50,100]
df['Age'] = pd.cut(df['Age'], bins=arr)
df_test['Age'] = pd.cut(df_test['Age'], bins=arr)

In [601]:
df.drop(columns='Age', inplace=True)
df_test.drop(columns='Age', inplace=True)

In [478]:
df['Clas_Gender']=df['Pclass'].apply(lambda x: np.exp(x))
df_test['Clas_Gender']=df_test['Pclass'].apply(lambda x: np.exp(x))

In [490]:
for i,r in df.iterrows():
    #df.loc[df.index[i],'Class_Gender']= df.loc[df.index[i],'Pclass']
    if(df.loc[df.index[i],'Pclass'] == 1):
        pw=3
    elif(df.loc[df.index[i],'Pclass'] == 2):
        pw=2
    else:
        pw=1
        
    if(df.loc[df.index[i],'Sex'] == 1):
        no = 10
    else:
        no = 2
        
    df.loc[df.index[i],'Clas_Gender'] = np.power( no , pw)

for i,r in df_test.iterrows():
    #df.loc[df.index[i],'Class_Gender']= df.loc[df.index[i],'Pclass']
    if(df_test.loc[df_test.index[i],'Pclass'] == 1):
        pw=3
    elif(df_test.loc[df_test.index[i],'Pclass'] == 2):
        pw=2
    else:
        pw=1
        
    if(df_test.loc[df_test.index[i],'Sex'] == 1):
        no=10
    else:
        no =2
        
    df_test.loc[df_test.index[i],'Clas_Gender'] = np.power(no , pw)

In [606]:
for label, content in df.items():
    if not pd.api.types.is_numeric_dtype(content):
        df[label] = content.astype("category").cat.as_ordered()

for label, content in df_test.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_test[label] = content.astype("category").cat.as_ordered()

In [607]:
for label, content in df.items():
    if not pd.api.types.is_numeric_dtype(content):
            df[label] = pd.Categorical(content).codes

for label, content in df_test.items():
    if not pd.api.types.is_numeric_dtype(content):
            df_test[label] = pd.Categorical(content).codes

In [572]:
df.drop(columns='Embarked', inplace=True)
df_test.drop(columns='Embarked', inplace=True)

In [579]:
df.drop(columns='Parch', inplace=True)
df_test.drop(columns='Parch', inplace=True)

df.drop(columns='Age', inplace=True)
df_test.drop(columns='Age', inplace=True)

df.drop(columns='SibSp', inplace=True)
df_test.drop(columns='SibSp', inplace=True)

df.drop(columns='Family', inplace=True)
df_test.drop(columns='Family', inplace=True)

In [604]:
df.drop(columns='SibSp', inplace=True)
df_test.drop(columns='SibSp', inplace=True)

In [669]:
df['Sex']=df['Sex']*4 + 1
df['m']=df['Pclass']*(df['Sex'])

df_test['Sex']=df_test['Sex']*4 + 1
df_test['m']=df_test['Pclass']*(df_test['Sex'])



In [670]:
df

,Survived,Pclass,Sex,Parch,Fare,Embarked,m
0,0,3,5,0,7.2500,2,15
1,1,1,1,0,71.2833,0,1
2,1,3,1,0,7.9250,2,3
3,1,1,1,0,53.1000,2,1
4,0,3,5,0,8.0500,2,15
...,...,...,...,...,...,...,...
886,0,2,5,0,13.0000,2,10
887,1,1,1,0,30.0000,2,1
888,0,3,1,2,23.4500,2,3
889,1,1,5,0,30.0000,0,5


In [707]:
np.random.seed(0)
valid = df.sample(n = int(len(df)*0.3))
train = df.drop(valid.index)
threshold= 0.5

In [672]:
df_X_train, df_Y_train, df_X_test, df_Y_test = prepareData(df,df_test,is_valid = False)
X_train, Y_train, X_test, Y_test = prepareData(train,valid,is_valid = True)

In [710]:
parameters = nn_model(df_X_train, df_Y_train,n_h=10, iter_num=50000,lr= 0.025,pr_cost=True)

(6, 891) (1, 891)
Cost after iteration 0: 0.693617
Cost after iteration 1000: 0.510621
Cost after iteration 2000: 0.461977
Cost after iteration 3000: 0.481860
Cost after iteration 4000: 0.456919
Cost after iteration 5000: 0.448607
Cost after iteration 6000: 0.448749
Cost after iteration 7000: 0.449719
Cost after iteration 8000: 0.444802
Cost after iteration 9000: 0.440428
Cost after iteration 10000: 0.437956
Cost after iteration 11000: 0.438803
Cost after iteration 12000: 0.437311
Cost after iteration 13000: 0.435999
Cost after iteration 14000: 0.432179
Cost after iteration 15000: 0.431843
Cost after iteration 16000: 0.429296
Cost after iteration 17000: 0.427609
Cost after iteration 18000: 0.426156
Cost after iteration 19000: 0.436360
Cost after iteration 20000: 0.445741
Cost after iteration 21000: 0.423535
Cost after iteration 22000: 0.422528
Cost after iteration 23000: 0.423111
Cost after iteration 24000: 0.422008
Cost after iteration 25000: 0.423845
Cost after iteration 26000: 0.420

In [694]:
parameters = nn_model(X_train, Y_train,n_h=10, iter_num=50000,lr= 0.02,pr_cost=True)

(6, 624) (1, 624)
Cost after iteration 0: 0.693584
Cost after iteration 1000: 0.489646
Cost after iteration 2000: 0.483552
Cost after iteration 3000: 0.485525
Cost after iteration 4000: 0.466825
Cost after iteration 5000: 0.475655
Cost after iteration 6000: 0.475622
Cost after iteration 7000: 0.455770
Cost after iteration 8000: 0.452399
Cost after iteration 9000: 0.448411
Cost after iteration 10000: 0.446404
Cost after iteration 11000: 0.444311
Cost after iteration 12000: 0.451015
Cost after iteration 13000: 0.437075
Cost after iteration 14000: 0.438656
Cost after iteration 15000: 0.451828
Cost after iteration 16000: 0.432179
Cost after iteration 17000: 0.432398
Cost after iteration 18000: 0.429057
Cost after iteration 19000: 0.427276
Cost after iteration 20000: 0.424918
Cost after iteration 21000: 0.423809
Cost after iteration 22000: 0.422036
Cost after iteration 23000: 0.421264
Cost after iteration 24000: 0.419939
Cost after iteration 25000: 0.421081
Cost after iteration 26000: 0.419

In [695]:
predictions = predict(parameters,X_test,threshold)
print ('Accuracy: %d' % float((np.dot(Y_test, predictions.T) + np.dot(1 - Y_test, 1 - predictions.T)) / float(Y_test.size) * 100) + '%')

Accuracy: 82%


C:\Users\bgraslan\AppData\Local\Temp\ipykernel_18888\620343402.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print ('Accuracy: %d' % float((np.dot(Y_test, predictions.T) + np.dot(1 - Y_test, 1 - predictions.T)) / float(Y_test.size) * 100) + '%')


In [696]:
predictions = predict(parameters,X_train,threshold)
print ('Accuracy: %d' % float((np.dot(Y_train, predictions.T) + np.dot(1 - Y_train, 1 - predictions.T)) / float(Y_train.size) * 100) + '%')

Accuracy: 81%


C:\Users\bgraslan\AppData\Local\Temp\ipykernel_18888\702157129.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print ('Accuracy: %d' % float((np.dot(Y_train, predictions.T) + np.dot(1 - Y_train, 1 - predictions.T)) / float(Y_train.size) * 100) + '%')


In [709]:
predictions = predict(parameters, df_X_train,threshold)
print ('Accuracy: %d' % float((np.dot(df_Y_train, predictions.T) + np.dot(1 - df_Y_train, 1 - predictions.T)) / float(df_Y_train.size) * 100) + '%')

Accuracy: 81%


C:\Users\bgraslan\AppData\Local\Temp\ipykernel_18888\1021066480.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print ('Accuracy: %d' % float((np.dot(df_Y_train, predictions.T) + np.dot(1 - df_Y_train, 1 - predictions.T)) / float(df_Y_train.size) * 100) + '%')


In [703]:
predictions = predict(parameters, df_X_test,threshold)

In [704]:
predictions = predictions.astype(int)

In [705]:
write= pd.read_csv("Data/test.csv")
result=pd.DataFrame({"PassengerId":write['PassengerId'],"Survived":predictions[0]})
result.to_csv("resultNN.csv", index=False)

In [706]:
predictions

array([[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
        1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 